In [ ]:
import pandas as pd
import numpy as np
from scipy.integrate import cumtrapz
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

In [ ]:
def plot_motion_path(df):
    # Make a copy of the dataframe to avoid modifying the original
    df_copy = df.copy()

    timestamps = pd.to_datetime(df['locationTimestamp_since1970(s)'], unit='s')
    time_diff = timestamps.diff()
    cumulative_seconds = time_diff.dt.total_seconds().cumsum().fillna(0)

    df_copy["Elapsed Time (s)"] = cumulative_seconds  
    time = df_copy['Elapsed Time (s)']

    df_copy['Acceleration X (m/s2)'] = df_copy['motionUserAccelerationX(G)'] * 9.80665
    df_copy['Acceleration Y (m/s2)'] = df_copy['motionUserAccelerationY(G)']* 9.80665
    df_copy['Acceleration Z (m/s2)'] = df_copy['motionUserAccelerationZ(G)']* 9.80665

    # Calculate velocity by integrating acceleration over time
    df_copy['Velocity X (m/s)'] = df_copy['Acceleration X (m/s2)'].cumsum() * df_copy['Elapsed Time (s)'].diff().fillna(0)
    df_copy['Velocity Y (m/s)'] = df_copy['Acceleration Y (m/s2)'].cumsum() * df_copy['Elapsed Time (s)'].diff().fillna(0)
    df_copy['Velocity Z (m/s)'] = df_copy['Acceleration Z (m/s2)'].cumsum() * df_copy['Elapsed Time (s)'].diff().fillna(0)

    # Calculate distance by integrating velocity over time
    df_copy['Distance X (m)'] = df_copy['Velocity X (m/s)'].cumsum() * df_copy['Elapsed Time (s)'].diff().fillna(0)
    df_copy['Distance Y (m)'] = df_copy['Velocity Y (m/s)'].cumsum() * df_copy['Elapsed Time (s)'].diff().fillna(0)
    df_copy['Distance Z (m)'] = df_copy['Velocity Z (m/s)'].cumsum() * df_copy['Elapsed Time (s)'].diff().fillna(0)

    x_dist = df_copy["Distance X (m)"]
    y_dist = df_copy["Distance Y (m)"]
    z_dist = df_copy["Distance Z (m)"]

    # Create a 3D plot
    fig = go.Figure(data=go.Scatter3d(
        x=x_dist,
        y=y_dist,
        z=z_dist,
        mode='markers',
        marker=dict(
            size=2,
            color=df['locationTimestamp_since1970(s)'],
            colorscale='Jet',
            colorbar=dict(thickness=20),
            opacity=0.8
        ),
        line=dict(color='darkblue', width=2)
    ))

    fig.show()


In [ ]:
def plot_3d_motion(data, start_time=None, end_time=None):
    df = data
    if start_time is not None and end_time is not None:
        # Subset the data between start and end times
        mask = (data.index > start_time) & (data.index <= end_time)
        df = data.loc[mask]

    # Create a new figure and add a 3D subplot.
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the 3D trajectory of motion.
    ax.plot3D(df['motionQuaternionX(R)'], 
              df['motionQuaternionY(R)'], 
              df['motionQuaternionZ(R)'], 'gray')

    # Provide labels
    ax.set_xlabel('Quaternion X')
    ax.set_ylabel('Quaternion Y')
    ax.set_zlabel('Quaternion Z')

    plt.show()

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(df['motionYaw(rad)'], df['motionRoll(rad)'], df['motionPitch(rad)'])
    ax.set_xlabel('Yaw (rad)')
    ax.set_ylabel('Roll (rad)')
    ax.set_zlabel('Pitch (rad)')

    plt.show()